# CS 439 Final Project: Crime Rate Prediction

**Contributers:** Jenya Pandu (jp2068), Vivian Zhao (vz57), Matt Yun (yy656) \
**Course:** Introduction to Data Science (01:198:439) \
**Professor:** Naina Chaturvedi \
**Semester:** Spring 2025 

### Import necessary libraries:

In [3]:

import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# geospatial / mapping
# import folium
# from folium.plugins import HeatMap

In [4]:

df = pd.read_csv("Chicage_Crime_Data.csv", low_memory=False)
print(df.shape)


(4380, 22)


In [34]:
df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,5741943,HN549294,08/25/2007 09:22:18 AM,074XX N ROGERS AVE,0560,ASSAULT,SIMPLE,OTHER,False,False,...,49.0,1.0,08A,NaN,NaN,2007.0,08/17/2015 03:03:40 PM,NaN,NaN,NaN
1,25953,JE240540,05/24/2021 03:06:00 PM,020XX N LARAMIE AVE,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,True,False,...,36.0,19.0,01A,1141387.0,1913179.0,2021.0,11/18/2023 03:39:49 PM,41.917838,-87.755969,"(41.917838056, -87.755968972)"
2,26038,JE279849,06/26/2021 09:24:00 AM,062XX N MC CORMICK RD,0110,HOMICIDE,FIRST DEGREE MURDER,PARKING LOT,True,False,...,50.0,13.0,01A,1152781.0,1941458.0,2021.0,11/18/2023 03:39:49 PM,41.995219,-87.713355,"(41.995219444, -87.713354912)"
3,13279676,JG507211,11/09/2023 07:30:00 AM,019XX W BYRON ST,0620,BURGLARY,UNLAWFUL ENTRY,APARTMENT,False,False,...,47.0,5.0,05,1162518.0,1925906.0,2023.0,11/18/2023 03:39:49 PM,41.952345,-87.677975,"(41.952345086, -87.677975059)"
4,13274752,JG501049,11/12/2023 07:59:00 AM,086XX S COTTAGE GROVE AVE,0454,BATTERY,"AGGRAVATED P.O. - HANDS, FISTS, FEET, NO / MIN...",SMALL RETAIL STORE,True,False,...,6.0,44.0,08B,1183071.0,1847869.0,2023.0,12/09/2023 03:41:24 PM,41.737751,-87.604856,"(41.737750767, -87.604855911)"


In [5]:
def preprocess(data):
    df = data.copy()
    
    df = df.drop_duplicates()
    df.dropna(subset=["Latitude", "Longitude", "Date", "Primary Type", "Description"], inplace=True)
    df = df.reset_index(drop=True)

    # remove unnecessary cols
    df = df.drop(columns=['ID', 'Case Number', 'Year',
                          'Block', 'IUCR', 'Description', 'Updated On',
                          'X Coordinate', 'Y Coordinate', 'Location'])   
    
    df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%Y %I:%M:%S %p", errors="coerce")
    df["Arrest"] = df["Arrest"].astype(bool)
    df["Domestic"] = df["Domestic"].astype(bool)
    df["Ward"] = df["Ward"].astype("Int64")

    df.info()
    
    return df

clean_df = preprocess(df)
display(clean_df)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4310 entries, 0 to 4309
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  4310 non-null   datetime64[ns]
 1   Primary Type          4310 non-null   object        
 2   Location Description  4299 non-null   object        
 3   Arrest                4310 non-null   bool          
 4   Domestic              4310 non-null   bool          
 5   Beat                  4310 non-null   float64       
 6   District              4310 non-null   float64       
 7   Ward                  4310 non-null   Int64         
 8   Community Area        4310 non-null   float64       
 9   FBI Code              4310 non-null   object        
 10  Latitude              4310 non-null   float64       
 11  Longitude             4310 non-null   float64       
dtypes: Int64(1), bool(2), datetime64[ns](1), float64(5), object(3)
memory usage:

,Date,Primary Type,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,Latitude,Longitude
0,2021-05-24 15:06:00,HOMICIDE,STREET,True,False,2515.0,25.0,36,19.0,01A,41.917838,-87.755969
1,2021-06-26 09:24:00,HOMICIDE,PARKING LOT,True,False,1711.0,17.0,50,13.0,01A,41.995219,-87.713355
2,2023-11-09 07:30:00,BURGLARY,APARTMENT,False,False,1922.0,19.0,47,5.0,05,41.952345,-87.677975
3,2023-11-12 07:59:00,BATTERY,SMALL RETAIL STORE,True,False,632.0,6.0,6,44.0,08B,41.737751,-87.604856
4,2023-09-06 17:00:00,CRIMINAL DAMAGE,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,122.0,1.0,42,32.0,14,41.886018,-87.633938
...,...,...,...,...,...,...,...,...,...,...,...,...
4305,2023-09-11 22:55:00,CRIMINAL DAMAGE,ALLEY,False,False,512.0,5.0,9,49.0,14,41.702951,-87.626972
4306,2023-09-12 00:00:00,BATTERY,APARTMENT,True,True,224.0,2.0,3,38.0,04B,41.806547,-87.622765
4307,2023-09-08 14:32:00,THEFT,APARTMENT,False,False,1424.0,14.0,1,24.0,06,41.909025,-87.687188
4308,2023-09-11 14:35:00,THEFT,RESIDENCE,False,False,231.0,2.0,3,40.0,06,41.794734,-87.620244
